In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config('spark.sql.warehouse.dir',f"/user/ubuntu/warehouse"). \
    config('spark.executor.instances',40). \
    enableHiveSupport(). \
    appName('warehouse creator'). \
    master('yarn'). \
    getOrCreate()

2023-01-23 06:56:14,903 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# executing one of the notebooks from the dataengineerin
# spark repo

In [4]:
spark

In [7]:
%%sh

ls /data/retail_db

categories
customers
departments
order_items
orders
products


The path has to be Hadoop file system

In [12]:
orders = spark.read.csv('/public/retail_db/orders/',
                        schema = """
                            order_id INT,
                            order_date STRING,
                            order_customer_id INT,
                            order_status STRING""")

In [11]:
%%sh

hdfs dfs -ls /public/retail_db

Found 6 items
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:02 /public/retail_db/categories
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:02 /public/retail_db/customers
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:03 /public/retail_db/departments
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:03 /public/retail_db/order_items
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:03 /public/retail_db/orders
drwxr-xr-x   - ubuntu supergroup          0 2023-01-21 15:03 /public/retail_db/products


In [13]:
orders.show(2)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
+--------+--------------------+-----------------+---------------+
only showing top 2 rows



In [15]:
%%sh 

hdfs dfs -ls /user/ubuntu/bigg_data

Found 1 items
drwxr-xr-x   - ubuntu supergroup          0 2023-01-22 13:53 /user/ubuntu/bigg_data/nyse_all


In [23]:
%%sh 

hdfs dfs -tail /user/ubuntu/bigg_data/nyse_all/nyse_stocks/companylist_noheader.csv

ttp://www.nasdaq.com/symbol/zmh|
ZB^A|Zions Bancorporation|20.8795|0|n/a|n/a|n/a|n/a|http://www.nasdaq.com/symbol/zb^a|
ZB^F|Zions Bancorporation|28.2|0|n/a|n/a|n/a|n/a|http://www.nasdaq.com/symbol/zb^f|
ZB^G|Zions Bancorporation|25.66|0|n/a|n/a|n/a|n/a|http://www.nasdaq.com/symbol/zb^g|
ZB^H|Zions Bancorporation|23.26|0|n/a|n/a|n/a|n/a|http://www.nasdaq.com/symbol/zb^h|
ZBK|Zions Bancorporation|27.0151|0|n/a|n/a|Finance|Major Banks|http://www.nasdaq.com/symbol/zbk|
ZOES|Zoe&#39;s Kitchen, Inc.|29.09|535103190.23|n/a|2014|Consumer Services|Restaurants|http://www.nasdaq.com/symbol/zoes|
ZTS|Zoetis Inc.|30.72|15391294156.8|n/a|2013|Health Care|Major Pharmaceuticals|http://www.nasdaq.com/symbol/zts|
ZA|Zuoan Fashion Limited|1.74|48405108.72|n/a|2011|Consumer Non-Durables|Apparel|http://www.nasdaq.com/symbol/za|
ZF|Zweig Fund, Inc. (The)|15.39|318197376.27|n/a|1986|n/a|n/a|http://www.nasdaq.com/symbol/zf|
ZTR|Zweig Total Return Fund, Inc. (The)|14.15|490203176.1|n/a|1988|n/a|n/a|http://www

In [30]:
nyse_read = spark.read.csv("/user/ubuntu/bigg_data/nyse_all/nyse_stocks/",
                          sep='|',inferSchema=True)
                           

In [31]:
nyse_read.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [5]:
spark.sql("USE practice_retail").show()

2023-01-23 06:56:51,766 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-01-23 06:56:51,767 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2023-01-23 06:56:53,534 ERROR metastore.ObjectStore: Version information found in metastore differs 1.2.0 from expected schema version 2.3.0. Schema verififcation is disabled hive.metastore.schema.verification
2023-01-23 06:56:53,535 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ubuntu@172.31.26.41


++
||
++
++



2023-01-23 06:56:53,750 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [6]:
spark.sql("SHOW TABLES")

database,tableName,isTemporary
practice_retail,order_items,false
practice_retail,orders,false


In [42]:
spark.sql("USE default")

""


In [43]:
spark.sql("SHOW TABLES")

database,tableName,isTemporary


In [35]:
%%sh

hdfs dfs -put /home/ubuntu/bigdata/yelp_academic_dataset_user.json \
    /user/ubuntu/bigg_data/yelp_data/

In [44]:
%%sh

hdfs dfs -ls /user/ubuntu/bigg_data/yelp_data/

Found 1 items
-rw-r--r--   1 ubuntu supergroup 2951073792 2023-01-23 06:11 /user/ubuntu/bigg_data/yelp_data/yelp_academic_dataset_user.json


In [45]:
yelping = spark.read.json("/user/ubuntu/bigg_data/yelp_data/")

In [9]:
%%sh

hdfs dfs -ls /user/ubuntu/bigg_data/airline/

Found 5 items
-rw-r--r--   1 ubuntu supergroup  792609318 2023-01-23 06:30 /user/ubuntu/bigg_data/airline/2009.csv
-rw-r--r--   1 ubuntu supergroup  794152224 2023-01-23 06:30 /user/ubuntu/bigg_data/airline/2010.csv
-rw-r--r--   1 ubuntu supergroup  746858940 2023-01-23 06:30 /user/ubuntu/bigg_data/airline/2011.csv
-rw-r--r--   1 ubuntu supergroup  775286574 2023-01-23 06:30 /user/ubuntu/bigg_data/airline/2012.csv
-rw-r--r--   1 ubuntu supergroup  121307136 2023-01-23 06:30 /user/ubuntu/bigg_data/airline/2013.csv


In [7]:
airline = spark.read.csv("/user/ubuntu/bigg_data/airline/2009.csv",
                         sep=',',inferSchema=True,header=True)
airline.show(2)

2023-01-23 06:57:25,580 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|

In [8]:
airline.count()

6429338

In [14]:
airline_schema = airline.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: do

In [11]:
airlineSchema = airline.schema

In [13]:
full_airline = spark.read.csv("/user/ubuntu/bigg_data/airline/",
                              schema=airlineSchema,
                              header=True)

In [14]:
full_airline.count()

26028000

In [20]:
full_airline.show(2)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|

In [25]:
from pyspark.sql.functions import *

In [29]:
full_airline = full_airline.drop('Unnamed: 27')

In [15]:
spark.sql("CREATE DATABASE airline_demo")

2023-01-23 07:02:30,484 WARN metastore.ObjectStore: Failed to get database airline_demo, returning NoSuchObjectException


""


In [16]:
spark.catalog.setCurrentDatabase('airline_demo')

In [17]:
spark.catalog.listDatabases()

[Database(name='airline_demo', description='', locationUri='hdfs://localhost:9000/user/ubuntu/warehouse/airline_demo.db'),
 Database(name='default', description='Default Hive database', locationUri='hdfs://localhost:9000/user/hive/warehouse'),
 Database(name='practice_retail', description='', locationUri='hdfs://localhost:9000/user/ubuntu/warehouse/practice_retail.db')]

In [18]:
spark.conf.set("spark.sql.shuffle.partitions","12")

In [30]:
full_airline.write.saveAsTable('full_airline',mode='overwrite')

2023-01-23 07:13:18,281 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2023-01-23 07:13:18,568 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2023-01-23 07:13:18,568 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-01-23 07:13:18,568 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [34]:
%%sh 

hdfs dfs -ls -h /user/ubuntu/warehouse/airline_demo.db/full_airline/

Found 26 items
-rw-r--r--   1 ubuntu supergroup          0 2023-01-23 07:13 /user/ubuntu/warehouse/airline_demo.db/full_airline/_SUCCESS
-rw-r--r--   1 ubuntu supergroup     23.3 M 2023-01-23 07:11 /user/ubuntu/warehouse/airline_demo.db/full_airline/part-00000-252ee949-904f-476f-bced-994be24cf212-c000.snappy.parquet
-rw-r--r--   1 ubuntu supergroup     23.3 M 2023-01-23 07:11 /user/ubuntu/warehouse/airline_demo.db/full_airline/part-00001-252ee949-904f-476f-bced-994be24cf212-c000.snappy.parquet
-rw-r--r--   1 ubuntu supergroup     23.3 M 2023-01-23 07:11 /user/ubuntu/warehouse/airline_demo.db/full_airline/part-00002-252ee949-904f-476f-bced-994be24cf212-c000.snappy.parquet
-rw-r--r--   1 ubuntu supergroup     23.4 M 2023-01-23 07:11 /user/ubuntu/warehouse/airline_demo.db/full_airline/part-00003-252ee949-904f-476f-bced-994be24cf212-c000.snappy.parquet
-rw-r--r--   1 ubuntu supergroup     23.1 M 2023-01-23 07:11 /user/ubuntu/warehouse/airline_demo.db/full_airline/part-00004-252ee949-904f-4